In [1]:
!pip install xgboost

  Using cached xgboost-3.1.3-py3-none-win_amd64.whl.metadata (2.0 kB)
Using cached xgboost-3.1.3-py3-none-win_amd64.whl (72.0 MB)


In [2]:
!pip install catboost

  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 1.0/102.4 MB 4.9 MB/s eta 0:00:21
    --------------------------------------- 2.1/102.4 MB 4.8 MB/s eta 0:00:22
   - -------------------------------------- 3.1/102.4 MB 4.8 MB/s eta 0:00:21
   - -------------------------------------- 4.2/102.4 MB 4.7 MB/s eta 0:00:21
   -- ------------------------------------- 5.2/102.4 MB 4.9 MB/s eta 0:00:20
   -- ------------------------------------- 6.3/102.4 MB 4.8 MB/s eta 0:00:20
   -- ------------------------------------- 7.6/102.4 MB 5.0 MB/s eta 0:00:19
   --- ------------------------------------ 8.7/102.4 MB 5.0 MB/s eta 0:00:19
   --- ------------------------------------ 9.4/102.4 MB 4.9 MB/s eta 0:00:19
   ---- ----------------------------------- 10.7/102.4 MB 5.0 MB/s eta 0:00:19
   ---- ------

In [3]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings